<a href="https://colab.research.google.com/github/nina-adhikari/enjoyment-maximizing-maps/blob/main/citydata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Data cleaning + preprocessing**
In this notebook, we import data from a few spreadsheets, strip it and combine it. The code here is written to run in Google Colab (files must be stored in the Google Drive folder drive/MyDrive/walkability) but it can be modified to run locally.

The datasets are:

*   **500 Cities: Local Data for Better Health**: This dataset contains estimates for 27 measures of chronic disease related to unhealthy behaviors (5), health outcomes (13), and use of preventive services (9). It includes estimates for the 500 largest US cities and approximately 28,000 census tracts within these cities.  Data sources used to generate these measures include Behavioral Risk Factor Surveillance System (BRFSS) data (2017, 2016), Census Bureau 2010 census population data, and American Community Survey (ACS) 2013-2017, 2012-2016 estimates. More information about the methodology can be found at www.cdc.gov/500cities. Link to dataset: https://data.cdc.gov/500-Cities-Places/500-Cities-Local-Data-for-Better-Health-2019-relea/6vp6-wxuq/about_data
*   **Smart Location Database (SLD)**: summarizes more than 90 different indicators associated with the built environment and location efficiency. Indicators include density of development, diversity of land use, street network design, and accessibility to destinations as well as various demographic and employment statistics. Most attributes are available for all U.S. block groups. Link to direct download of dataset: https://edg.epa.gov/EPADataCommons/public/OA/EPA_SmartLocationDatabase_V3_Jan_2021_Final.csv
*   **Median household income** Median household income in the past 12 months (in 2022 inflation-adjusted dollars), for the District of Columbia only, year 2017. Link to dataset:https://data.census.gov/table/ACSDT5Y2017.B19013?q=b19013&g=040XX00US11$1400000&moe=false&tp=true

We focus our analysis in the **District of Columbia only**. To do so, it is neccesary to filter out other cities. We also filter out factors that we do not wish to include in the analysis.



In [ ]:
#import packages to handle data
import geopandas as gpd #extends the datatypes used by pandas to allow spatial operations on geometric types
import pandas as pd
!pip install mapclassify #Choropleth map classification

In [5]:
#allow colab to access your Google Drive, an authentication window will pop-up
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [7]:
#import datasets stored in drive/MyDrive/walkability/
health = 'drive/MyDrive/walkability/health.csv'      # 500 Cities: Local Data for Better Health https://data.cdc.gov/500-Cities-Places/500-Cities-Local-Data-for-Better-Health-2019-relea/6vp6-wxuq/about_data
epacsv = 'drive/MyDrive/walkability/epdownload.csv'  # Smart Location Database https://edg.epa.gov/EPADataCommons/public/OA/EPA_SmartLocationDatabase_V3_Jan_2021_Final.csv
incomesource = 'drive/MyDrive/walkability/ACSDT5Y2017.B19013-2024-03-26T221220.csv'      # Median household income https://data.census.gov/table/ACSDT5Y2017.B19013?q=b19013&g=040XX00US11$1400000&moe=false&tp=true
income = 'drive/MyDrive/walkability/income.csv'      # we will store the modified csv here

Clean up income dataset. Important: the ID of each row, for all files, is to be the census tract ID.

In [ ]:
#reformat the income file to have two columns: tract and income
with open(incomesource, 'r') as incomefile:
  #remove all unnecessary wording and spaces
  incomestr = incomefile.read().replace(
      ', District of Columbia, District of Columbia",""\n"    Estimate', '').replace(
          'Median household income in the past 12 months (in 2017 inflation-adjusted dollars)', 'Income').replace(
              '\ufeff"Label (Grouping)"', 'Tract').replace(
                  'Census Tract ', '')

#write reformatted data into new file
with open(income, 'w') as incomefile:
  incomefile.write(incomestr)

inc = gpd.read_file(income).drop('geometry', axis=1) #converts GeoDataFrame to a normal DataFrame by dropping the geometry column?
inc['Income'] = inc['Income'].str.replace(',','') #replace comma format for numbers
inc.apply(pd.to_numeric, errors='coerce') #convert to numeric, coerce makes invalid parsing be set as NaN
inc['Tract'] = round(inc['Tract'].astype(float)*100).astype(int) #format census tract id as integer
inc.set_index('Tract', inplace=True) #tract id is the index of this dataframe
inc #sanity check

Clean up the SLD dataset. We are keeping only the rows corresponding to the District of Columbia (state FIPS code 11), and the columns that are not derived from other variables in the dataset (see Documentation: https://www.epa.gov/system/files/documents/2023-10/epa_sld_3.0_technicaldocumentationuserguide_may2021_0.pdf)

In [ ]:
columns_to_drop = ['D1A', 'D1B', 'D1C', 'D1C5_RET', 'D1C5_OFF', 'D1C5_IND', 'D1C5_SVC',
                   'D1C5_ENT', 'D1C8_RET', 'D1C8_OFF', 'D1C8_IND', 'D1C8_SVC', 'D1C8_ENT',
                   'D1C8_ED', 'D1C8_HLTH', 'D1C8_PUB', 'D1D', 'D1_FLAG', 'D2A_JPHH',
                   'D2B_E5MIX', 'D2B_E5MIXA', 'D2B_E8MIX', 'D2B_E8MIXA', 'D2A_EPHHM',
                   'D2C_TRPMX1', 'D2C_TRPMX2', 'D2C_TRIPEQ', 'D2R_JOBPOP', 'D2R_WRKEMP',
                   'D2A_WRKEMP', 'D2C_WREMLX', 'D4D', 'D4E', 'D5CR', 'D5CRI', 'D5CE',
                   'D5CEI', 'D5DR', 'D5DRI', 'D5DE', 'D5DEI',
                   'D2A_Ranked', 'D2B_Ranked', 'D3B_Ranked', 'D4A_Ranked'
                   ]
epa = gpd.read_file(epacsv, where="STATEFP='11'").drop(columns=columns_to_drop)   # only interested in DC
epa['TRACTCE'] = epa['TRACTCE'].astype(int) #this will be the index, format as integer
epa.set_index('OBJECTID', inplace=True)
epa

In [ ]:
#gdb = 'drive/MyDrive/smartlocation/smartlocdb/SmartLocationDatabase.gdb'
#fc = gpd.read_file(gdb, where="STATEFP='11'")

Finally, clean up the Local Data for Better Health dataset. Keep also only DC.

In [ ]:
hl = gpd.read_file(health, where="StateAbbr='DC' AND GeographicLevel='Census Tract'", include_fields=['StateAbbr', 'GeographicLevel', 'UniqueID', 'MeasureId', 'CityFIPS', 'TractFIPS', 'Data_Value'])
hl['TractCE'] = hl['TractFIPS'].str.removeprefix('110010').astype(int) #format index column
hlp = hl.pivot(index='TractCE', columns='MeasureId', values='Data_Value') #pivot table to have the columns be the MeasureId
hlp

Finally, merge the datasets on the Census tract FIPS code.

In [ ]:
firstjoin = epa.join(other=hlp, on='TRACTCE', how='left', validate='m:1')           # merge EPA and CDC
secondjoin = firstjoin.join(other=inc, on='TRACTCE', how='left', validate='m:1')    # merge above with census income data # m:1 checks if join keys are unique in right dataset
secondjoin

Print resulting columns, and create a dictionary for the columns.

In [12]:
print('there are',len(secondjoin.columns.to_list()),'columns')
print(secondjoin.columns.to_list())

there are 101 columns
['GEOID10', 'GEOID20', 'STATEFP', 'COUNTYFP', 'TRACTCE', 'BLKGRPCE', 'CSA', 'CSA_Name', 'CBSA', 'CBSA_Name', 'CBSA_POP', 'CBSA_EMP', 'CBSA_WRK', 'Ac_Total', 'Ac_Water', 'Ac_Land', 'Ac_Unpr', 'TotPop', 'CountHU', 'HH', 'P_WrkAge', 'AutoOwn0', 'Pct_AO0', 'AutoOwn1', 'Pct_AO1', 'AutoOwn2p', 'Pct_AO2p', 'Workers', 'R_LowWageWk', 'R_MedWageWk', 'R_HiWageWk', 'R_PCTLOWWAGE', 'TotEmp', 'E5_Ret', 'E5_Off', 'E5_Ind', 'E5_Svc', 'E5_Ent', 'E8_Ret', 'E8_off', 'E8_Ind', 'E8_Svc', 'E8_Ent', 'E8_Ed', 'E8_Hlth', 'E8_Pub', 'E_LowWageWk', 'E_MedWageWk', 'E_HiWageWk', 'E_PctLowWage', 'D3A', 'D3AAO', 'D3AMM', 'D3APO', 'D3B', 'D3BAO', 'D3BMM3', 'D3BMM4', 'D3BPO3', 'D3BPO4', 'D4A', 'D4B025', 'D4B050', 'D4C', 'D5AR', 'D5AE', 'D5BR', 'D5BE', 'NatWalkInd', 'Shape_Length', 'Shape_Area', 'geometry', 'ACCESS2', 'ARTHRITIS', 'BINGE', 'BPHIGH', 'BPMED', 'CANCER', 'CASTHMA', 'CHD', 'CHECKUP', 'CHOLSCREEN', 'COLON_SCREEN', 'COPD', 'COREM', 'COREW', 'CSMOKING', 'DENTAL', 'DIABETES', 'HIGHCHOL', '

In [ ]:
#create dictionary of columns
cols_dict_data = 'drive/MyDrive/walkability/column-dictionary.csv'

# import csv
import csv

#create dictionary with the meaning of the cols names
with open(cols_dict_data, mode='r') as infile:
    reader = csv.reader(infile)
    cols_dict = {rows[0]:rows[1] for rows in reader}

cols_dict